In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/MIS584/')

In [ ]:
!pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import seaborn as sns
%pylab inline
pd.set_option('display.max_columns',None)
pd.options.display.max_seq_items = 2000
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import requests, re
import pandas as pd
import seaborn as sns
import nltk
import string, itertools
from collections import Counter, defaultdict
from nltk.text import Text
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from sklearn.cluster import KMeans
from wordcloud import WordCloud
import json
import numpy as np 
import pandas as pd 

import tensorflow.keras.metrics
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import sklearn.metrics
import re

Populating the interactive namespace from numpy and matplotlib


In [ ]:
data_file = open("yelp_academic_dataset_review.json")
datas = []
for line in data_file:
  if json.loads(line)['date'] >= "2021-01-01 00:00:00" and json.loads(line)['date'] < "2021-02-01 00:00:00":
    datas.append(json.loads(line))
review_df = pd.DataFrame(datas)
data_file.close()

In [ ]:
review_df.count() #618189
#review_df.head(2)

review_id      45490
user_id        45490
business_id    45490
stars          45490
useful         45490
funny          45490
cool           45490
text           45490
date           45490
dtype: int64

In [ ]:
data = review_df[['text', 'stars']]

In [ ]:
data['sentiment']=['pos' if (x>3) else 'neg' for x in data['stars']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data['text']= [x.lower() for x in data['text']]
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [ ]:
pd.set_option('display.max_colwidth',-1)
data[:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,text,stars,sentiment
0,ive been eating at this restaurant for over 5 years and it is a staple in me abs my families lives really good quality and even better prices i cant eat here without getting the gyoza the sauce is so good i could drink it i dont typically order crunchy rolls but theirs isnt a traditional roll with snow crab and crunchies theirs has spicy crunchy shrimp and crab stick delicious little spot,5.0,pos
1,how does a delivery person from here get lost have they not heard of google maps our food was room temp on the verge of cold by the time we received it an hour and a half after ordering will not be ordering again what happened to melting cheese back in the day now with excellent navigation we get cold food,1.0,neg
2,the service is always good the employees are nice and vegan african food i loveeeeee and the portions so sturdy,5.0,pos
3,two words whipped feta \nexplosion of amazingness hit your taste buds i seriously go just for this but their appetizer small plates split between a group is a great way to try a bunch of their food all at once also just a cozy atmosphere \nbut again whipped feta warm pita,5.0,pos
4,place was great as well as parking \nfood was good as well although they need a better vent so it doesnt get too smoky inside they had to open the front door which made it cold,4.0,pos


In [ ]:
data.dtypes

text         object 
stars        float64
sentiment    object 
dtype: object

In [ ]:
data['stars']=data['stars'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data['text']= [x.encode('utf-8') for x in data['text']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
tokenizer = Tokenizer(nb_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts([x.decode('utf-8') for x in data['text']])

/usr/local/lib/python3.7/dist-packages/keras/preprocessing/text.py:234: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [ ]:
tokenizer

In [ ]:
X = tokenizer.texts_to_sequences([x.decode('utf-8') for x in data['text']])
X = pad_sequences(X)

In [ ]:
len(data)

45490

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa
f1 = tfa.metrics.F1Score(num_classes=2, average=None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
embed_dim = 128
lstm_out = 196
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_out))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'F1Score', 'precision', 'recall'])
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1, tensorflow.metrics.Precision(), tensorflow.metrics.Recall()])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 892, 128)          256000    
                                                                 
 lstm_2 (LSTM)               (None, 196)               254800    
                                                                 
 dropout_2 (Dropout)         (None, 196)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(36392, 892) (36392, 2)
(9098, 892) (9098, 2)


In [ ]:
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 1, verbose = 2)

1138/1138 - 3084s - loss: 0.2617 - accuracy: 0.8937 - f1_score: 0.8702 - precision_2: 0.8937 - recall_2: 0.8937 - 3084s/epoch - 3s/step


In [ ]:
model

In [ ]:
score = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)

print("Validation Accuracy: %.2f" % (score[0]))
print("Score: %.2f" % (score[1]))
print("precision: " % (score[2]))
print("recall: %.2f" % (score[3]))

285/285 - 212s - loss: 0.2295 - accuracy: 0.9129 - f1_score: 0.8937 - precision_2: 0.9129 - recall_2: 0.9129 - 212s/epoch - 744ms/step
Validation Accuracy: 0.23
Score: 0.91
precision: 
recall: 0.91


In [ ]:
#Get the predictions

y_pred = model.predict(X_test)

285/285 [==============================] - 231s 800ms/step


In [ ]:
y_pred=np.argmax(y_pred, axis=1)
Y_test=np.argmax(Y_test, axis=1)

from sklearn.metrics import confusion_matrix

#Generate the confusion matrix
cf_matrix = confusion_matrix(Y_test, y_pred)

print(cf_matrix)

[[2217  503]
 [ 289 6089]]


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
y_pred1=np.argmax(y_pred, axis=1)
Y_test1=np.argmax(Y_test, axis=1)

print('Precision: %.3f' % precision_score(Y_test1, y_pred1))
print('Recall: %.3f' % recall_score(Y_test1, y_pred1))
print('Accuracy: %.3f' % accuracy_score(Y_test1, y_pred1))
print('F1 Score: %.3f' % f1_score(Y_test1, y_pred1))

Precision: 0.924
Recall: 0.955
Accuracy: 0.913
F1 Score: 0.939
